In [ ]:
from features import *
import librosa
import librosa.display as display
import scipy
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from model_02_functions import *
import sklearn
from train_period_checker import max_pooling
import matplotlib.colors as colors
from timetables import trains_in_period
from wavfile_manipulations import extract_wavfile_and_date
from octave_band import *
from initial_functions import *

In [ ]:
params = [1414,13000,80000,140000,380000,800000,1200000]
acc = [0.8,0.84,0.845,0.855,0.87,0.871,0.871]
fig = plt.figure(figsize=(8,4.5))
plt.plot(params, acc, color='#e32017')
plt.xlim
plt.xscale('log')
plt.xlabel('No. Parameters')
plt.ylabel('Average Accuracy')
plt.savefig('final_figures/further/simplecomplex',dpi=200)
plt.show()

In [ ]:
import os
import pandas as pd
from scipy.io import wavfile
from scipy.fftpack import fft
from scipy.fftpack import ifft
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
import scipy
from scipy.signal import welch
import librosa
import librosa.display
from tqdm import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
import keras
from keras.models import Sequential
from keras import Input, Model, layers
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, ReLU, AveragePooling2D, Add
from keras.optimizers import Adam, SGD
from keras.utils import np_utils
from sklearn import metrics 
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 
from model_02_functions import find_train_trainbar_array_datetime, rolling_spectrograms_with_labels_02
import random

In [ ]:
model = keras.models.load_model('network5_model.h5')
model.load_weights('network5_weights.h5')

In [ ]:
store = pd.HDFStore('initial_datasets/testing_hour13.h5')
#store.close()
test = store['first']

In [ ]:
store = pd.HDFStore('initial_datasets/spectro3.h5')
#store.close()
data = store['first']

In [ ]:
train = [1 if i==0 else 0 for i in data.no_train]
#data['no_train'] = no_train

In [ ]:
len(data.array[0])

In [ ]:
final = pd.concat([data[data.no_train==0],data[data.no_train==1].sample(300)],ignore_index=True).sample(frac=1).reset_index(drop=True)

In [ ]:
X = np.array(final.array.tolist())

In [ ]:
y = np.array(final.drop(['array'],axis=1))

In [ ]:
y

In [ ]:
count=0
for i in y.argmax(axis=1):
    if i==4:
        count+=1
print(count)

In [ ]:
pred = model.predict(X[0:500])
pred=np.transpose(pred)

In [ ]:
pred

In [ ]:
pred = pred[0]

In [ ]:
def expand_singles(array):
    output = []
    for i in array:
        if i==1:
            for j in range(5):
                output.append(1)
        else:
            for j in range(5):
                output.append(0)
    return output

In [ ]:
data13, time = extract_wavfile_and_date('train_recordings/hour1301.wav', samplerate=3200)

In [ ]:
times = trains_in_period(['cen_eas','cen_wes','bak_nor','bak_sou','vic_nor','vic_sou'],time)

In [ ]:
print(times)

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = moving_average(abs(return_octave_filtered(data13,samplerate=3200,octave_band=125)),1500)
t = np.linspace(0,3600,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($V$)')
plt.xlim([0,600])
plt.ylim([0,0.0015])
plt.plot(t,train_audio,color='#0098D4',lw=0.3)
"""for i in times:
    for j in times[i]:
        plt.axvline(j,color='#E32017',lw=2.5,alpha=1)
plt.axvline(times['cen_eas'][0],color='#E32017',lw=2.5,alpha=1,label='Expected Train')"""
plt.plot(np.linspace(0,3600,718),[0.001*i for i in pred], color='#E32017')
#plt.legend(loc='upper right')
plt.savefig('final_figures/approach/CNN',dpi=200)
plt.show()

In [ ]:
matrix = metrics.confusion_matrix(y[500:1000].argmax(axis=1),pred.argmax(axis=1))

In [ ]:
matrix

In [ ]:
import seaborn as sns

In [ ]:
cf_matrix = [[586,250],[222,389]]
fig = plt.figure(figsize=(8,4.5))
x_label = ['Positive','Negative']
sns.heatmap(cf_matrix, annot=True,cmap='Blues',xticklabels=x_label,yticklabels=x_label,cbar=False,fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('final_figures/intro/confusion',dpi=200)
plt.show()

In [ ]:
def find_metrics(c):
    acc = (c[0]+c[1])/(sum(c))
    pre = c[0]/(c[0]+c[2])
    rec = c[0]/(c[0]+c[3])
    f1 = 2*pre*rec/(pre+rec)
    return [acc,pre,rec,f1]

In [ ]:
print(find_metrics([187,120,58,353]))

# next

In [ ]:
model = keras.models.load_model('networkreallysimple_model.h5')
model.load_weights('networkreallysimple_weights.h5')

In [ ]:
pred = model.predict(X[0:500])
pred=np.transpose(pred)
pred=pred[0]

In [ ]:
matrix = metrics.confusion_matrix(y[0:500].argmax(axis=1),pred.argmax(axis=1))
matrix

In [ ]:
fig = plt.figure(figsize=(8,4.5))
x_label = ['cen_eas','cen_wes','bak_sou','bak_nor','vic_sou','vic_nor','no train']
sns.heatmap(matrix, annot=True,cmap='Blues',xticklabels=x_label,yticklabels=x_label,cbar=False,fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('final_figures/further/simpleconfusion',dpi=200)
plt.show()

In [ ]:
model = keras.models.load_model('networkcomplex_model.h5')
model.load_weights('network7complexweights.h5')
pred = model.predict(X[0:500])
pred=np.transpose(pred)
pred=pred[0]
matrix = metrics.confusion_matrix(y[:500].argmax(axis=1),pred.argmax(axis=1))

In [ ]:
fig = plt.figure(figsize=(8,4.5))
x_label = ['cen_eas','cen_wes','bak_sou','bak_nor','vic_sou','vic_nor','no train']
sns.heatmap(matrix, annot=True,cmap='Blues',xticklabels=x_label,yticklabels=x_label,cbar=False,fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('final_figures/further/complexconfusion',dpi=200)
plt.show()

In [ ]:
model = keras.models.load_model('network7_model.h5')
model.load_weights('network7_weights.h5')
pred = model.predict(X[0:500])
pred=np.transpose(pred)
pred=pred[0]
matrix = metrics.confusion_matrix(y[:500].argmax(axis=1),pred.argmax(axis=1))

In [ ]:
matrix

In [ ]:
fig = plt.figure(figsize=(8,4.5))
x_label = ['cen_eas','cen_wes','bak_sou','bak_nor','vic_sou','vic_nor','no train']
sns.heatmap(matrix, annot=True,cmap='Blues',xticklabels=x_label,yticklabels=x_label,cbar=False,fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('final_figures/further/confusion7',dpi=200)
plt.show()

In [ ]:
model = keras.models.load_model('networkfinal_model.h5')
model.load_weights('networkfinal_weights.h5')
pred = model.predict(X[0:500])
pred=np.transpose(pred)
pred=pred[0]
matrix = metrics.confusion_matrix(y[:500].argmax(axis=1),pred.argmax(axis=1))

In [ ]:
fig = plt.figure(figsize=(8,4.5))
x_label = ['cen_eas','cen_wes','bak_sou','bak_nor','vic_sou','vic_nor','no train']
sns.heatmap(matrix, annot=True,cmap='Blues',xticklabels=x_label,yticklabels=x_label,cbar=False,fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('final_figures/further/confusionfinal',dpi=200)
plt.show()

In [ ]:
def find_metrics(c):
    acc = (c[0]+c[1])/(sum(c))
    pre = c[0]/(c[0]+c[2])
    rec = c[0]/(c[0]+c[3])
    f1 = 2*pre*rec/(pre+rec)
    return [acc,pre,rec,f1]

In [ ]:
conf = [405,42,5,16]
print(find_metrics(conf))

In [ ]:
conf = [415,40,15,12]
print(find_metrics(conf))

# SBD

In [ ]:
sbd = pd.read_csv('sbd126.csv')
sbd

In [ ]:
sbd = pd.read_csv('SBD/sbd240.csv')
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd241.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd242.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd243.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd244.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd245.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd246.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd247.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd248.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd249.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd250.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd251.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd252.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd253.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd254.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd255.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd256.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd257.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd258.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd259.csv')],axis=0)
sbd = pd.concat([sbd,pd.read_csv('SBD/sbd260.csv')],axis=0)

In [ ]:
sbd

In [ ]:
4888576/2048

In [ ]:
plt.plot(sbd['Column4'])
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = moving_average(abs(return_octave_filtered(sbd['Column4'],samplerate=2048,octave_band=125)),1500)
t = np.linspace(0,2387,len(train_audio))
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($V$)')
#plt.xlim([0,600])
#plt.ylim([0,0.0015])
plt.plot(t,train_audio,color='#0098D4',lw=0.3)

plt.legend(loc='upper right')
#plt.savefig('final_figures/further/predictfinal',dpi=200)
plt.show()

In [ ]:
def rolling_spectrograms_no_labels(data, leng, samplerate=1000, interval=10, lag=5):
    #empty array for spectrograms and whether a train
    output = []
    train = []

    #find start times for spectrograms
    starts = range(0,samplerate*(leng-interval),samplerate*lag)
 
    for start in tqdm(starts):
        end = start + interval*samplerate    
        f0, t0, feature = scipy.signal.spectrogram(data[start:end], fs=samplerate, window='hamming', nperseg=39, noverlap=0, nfft=640)
        feature = max_pooling(feature, pool_size=(3,3), pool_overlap=(1,1))
        lim = [(50*len(feature))//(samplerate/2), (300*len(feature))//(samplerate/2)]
        feature = feature[int(lim[0]):int(lim[1])]
        feature = [i[:125] for i in feature]
        #feature = [i[:128] for i in librosa.feature.melspectrogram(y=data[start:end], sr=samplerate,n_fft=100,hop_length=78)]
        #feature = librosa.feature.mfcc(y=data[start:end],sr=samplerate,n_mfcc=100, n_mels=100, hop_length=101, fmin=0, fmax=None, htk=False)
        output.append(feature)

    #join spectrograms to labels
    output = pd.Series(normalise(output), name='array')

    print(output.shape)
    print(len(output.array[0]))
    print(len(output.array[0][0]))

    return output

In [ ]:
new = rolling_spectrograms_no_labels(sbd['Column4'][:4884480],leng=1385,samplerate=2048)

In [ ]:
new=new.to_list()

In [ ]:
model = keras.models.load_model('networkfinal_model.h5')
model.load_weights('networkfinal_weights.h5')
pred = model.predict(new)
pred=np.transpose(pred)
pred=pred[0]

In [ ]:
pred

In [ ]:
print(pred.argmax(axis=1))

In [ ]:
fig = plt.figure(figsize=(8,4.5))
train_audio = moving_average(abs(return_octave_filtered(sbd['Column4'],samplerate=2048,octave_band=125)),1500)
t = np.linspace(0,2387,len(train_audio))
a = 2387/275
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($V$)')
plt.xlim([1500,2387])
#plt.xticks([0,200,400,600,800,1000])
plt.ylim([0,0.0002])
plt.plot(t,train_audio,color='#0098D4',lw=0.3)

for i,j in enumerate(pred.argmax(axis=1)):
    if j!=6:
        plt.fill_between(np.linspace(i*a,a*(i+1),2),[0.00025]*2, color='#e32017',alpha=0.4)
plt.fill_between(np.linspace(0,2,2),[0.00025]*2, color='#e32017',alpha=0.4,label='Predicted Train')
plt.legend(loc='upper right')
plt.savefig('final_figures/further/sbdpredict',dpi=200)
plt.show()

In [ ]:
def rolling_output_with_labels_02(hours, leng, samplerate=2048, interval=10, lag=5, ste_octave=None):
    #empty array for spectrograms and whether a train
    zcr = []
    ste = []
    rms = []
    peak = []
    cent = []
    rolloff = []

    #find start times for spectrograms
    starts = range(0,samplerate*(leng-interval),samplerate*lag)

    #loop through trains_array
    #feature = librosa.feature.mfcc(y=data[start:end], sr=samplerate, n_mfcc=40, n_mels=40, hop_length=160, fmin=0, fmax=None, htk=False)
    for start in tqdm(starts):
        end = start + interval*samplerate    
        zcr.append(zero_crossing_rate(data[start:end], samplerate=samplerate, frame_length=samplerate*interval)[0])
        ste.append(short_time_energy(data[start:end], samplerate=samplerate, frame_length=samplerate*interval)[0])
        rms.append(RMS(data[start:end], frame_length=samplerate*interval)[0])
        peak.append(peak_value(data[start:end], samplerate=samplerate, frame_length=samplerate*interval)[0])
        cent.append(spectral_centroid(data[start:end], samplerate=samplerate, frame_length=samplerate*interval)[0])
        rolloff.append(spectral_roll_off(data[start:end], samplerate=samplerate, frame_length=samplerate*interval)[0])

    #join spectrograms to labels

    output = pd.Series(normalisei(zcr), name='zcr')
    output = pd.concat([output, pd.Series(normalisei(ste), name='ste')],axis=1)
    output = pd.concat([output, pd.Series(normalisei(rms), name='rms')],axis=1)
    output = pd.concat([output, pd.Series(normalisei(peak), name='peak')],axis=1)
    output = pd.concat([output, pd.Series(normalisei(cent), name='cent')],axis=1)
    output = pd.concat([output, pd.Series(normalisei(rolloff), name='rolloff')],axis=1)

    print(output.shape)

    return output